In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


batch_size = 32

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_train)

trainset, validationset = random_split(dataset, [40000, 10000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def compute_accuracy(net, dataloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    
    for data in dataloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def numel(m: torch.nn.Module, only_trainable: bool = False):
    """
    returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = m.parameters()
    if only_trainable:
        parameters = list(p for p in parameters if p.requires_grad)
    unique = dict((p.data_ptr(), p) for p in parameters).values()
    return sum(p.numel() for p in unique)

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3,padding=1,stride=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128,128,3,padding=1,stride=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128*8*8, 600)
        self.bn2 = nn.BatchNorm1d(600)
        self.fc3 = nn.Linear(600, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu((self.bn2(self.fc1(x))))
        x = self.fc3(x)
        return x

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

net = Net2().to(device)
#4,6,8
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())#, #momentum=0.9)

epochs_done = 0

print(numel(net)/10**6)

train_accuracy = []
validation_accuracy = []

validation_loss = []
train_loss = []


In [ ]:
for epoch in range(150):  # loop over the dataset multiple times
    epochs_done += 1 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # wrap them in Variable
        #inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if epochs_done % 10 == 0 and epochs_done!=0 :
        torch.save(net.state_dict(), f"./res_net_shallow10M{epochs_done}")
    
    if epoch % 5 == 0:
        with torch.no_grad(): 
            v_loss = 0.0
            for j, data in enumerate(validationloader, 0):
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = net(inputs)
                v_loss += criterion(outputs, labels).item()
                
            validation_loss.append(v_loss/j)
            
            v_accuracy = compute_accuracy(net, validationloader)
            validation_accuracy.append(v_accuracy)
            
            train_loss.append(running_loss/i)
            t_accuracy = compute_accuracy(net, trainloader)
            train_accuracy.append(t_accuracy)
    
    print(f"Epoch: {epoch}, training loss: {running_loss/i}, validation loss: {v_loss/j}, validation accuracy: {v_accuracy}%")
                
    
                